In [1]:
import pandas as pd
import numpy as np
import requests
import yfinance as yf

import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot

In [2]:
response = requests.get("https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv")
response

<Response [200]>

In [3]:
response = response.content.decode("utf-8")
result = [x.split(',') for x in response.split('\n')]
df = pd.DataFrame(result[1:], columns= result[0])

In [4]:
# Substituindo os valores vazios por 0:
df = df.replace("", 0, regex=True)

In [6]:
#  Removendo colunas com valores constantes:
df.drop(["country", "city"], axis=1, inplace=True)
df = df[:-1]

KeyError: "['country', 'city'] not found in axis"

In [ ]:
# Transformando a coluna de data para o formato apropriado:
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")

In [ ]:
# Descobrindo quandos dias forama até a covid chegar no Brasil
PRIMEIRO_CASO_COVID_MUDO = pd.to_datetime('2019-11-17', format="%Y-%m-%d")
primeiro_caso_covid_brasil = df['date'][0]
dias = abs((PRIMEIRO_CASO_COVID_MUDO - primeiro_caso_covid_brasil).days)
dias

100

In [7]:
# Como a maioria das colunas está em um formato numérico, vamos converter elas para float:
for column in df.columns:
    try:
        df[column] = df[column].astype(float)
    except:
        continue

In [8]:
# Colocando as semanas epidemiológicas na ordem direta:
for row in df.index:
    if df["epi_week"][row] > 200:
        df["epi_week"][row] = (df["epi_week"][row]-200+105)
    elif df["epi_week"][row] > 100:
        df["epi_week"][row] = (df["epi_week"][row]-100+53)
        
# Acabei nem usando isso aqui

C:\Users\Giga\AppData\Local\Temp\ipykernel_38092\725959677.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Giga\AppData\Local\Temp\ipykernel_38092\725959677.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
# Separando os dados a nível nacional:
df_total = df[df["state"]=="TOTAL"]

In [10]:
# MANO, SALVA ESSA LINHA DE CÓDIGO PARA A VIDA:
# PARA A VIDA!     - Tiago
months = df_total.set_index("date").resample('M').sum()
months.head()

"""
Fazendo essa agregação não podemos trabalhar com colunas de valores agregados!!!! 
Por exemplo, a coluna deaths soma diversas vezes a coluna newDeaths, 
e quando ela é agregada ela soma esses valores mais de uma vez!
Isso gera um número absurdo de casos, além de dados errados!
Recomendo usar as colunas newDeaths e newCases da base de dados original,
as outras tem que ser avaliadas antes de serem usadas!!
"""

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [42]:
"""
Ao invés de usar resample poderiamos pensar em usar groupby, mas isso dificultaria - muito - a plotagem.

Groupby com multy index:
monthly_deaths = df_total.groupby([df_total.date.dt.year, df_total.date.dt.month])['newDeaths'].sum()

Groupby com index resetado:
monthly_deaths = df_total.groupby([df_total.date.dt.year, df_total.date.dt.month], as_index=False)['newDeaths'].sum()


""";

In [43]:
months

,epi_week,newDeaths,deaths,newCases,totalCases,deathsMS,totalCasesMS,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,...,tests,tests_per_100k_inhabitants,vaccinated,vaccinated_per_100_inhabitants,vaccinated_second,vaccinated_second_per_100_inhabitants,vaccinated_single,vaccinated_single_per_100_inhabitants,vaccinated_third,vaccinated_third_per_100_inhabitants
date,,,,,,,,,,,,,,,,,,,,,
2020-02-29,45.0,0.0,0.0,2.0,7.000000e+00,0.0,5.000000e+00,0.00000,0.00329,0.00000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.00000,0.000000e+00,0.00000,0.0,0.00000,0.000000e+00,0.00000
2020-03-31,364.0,202.0,998.0,5822.0,3.827900e+04,981.0,3.714400e+04,0.46786,17.94463,0.30899,...,0.000000e+00,0.000000e+00,0.000000e+00,0.00000,0.000000e+00,0.00000,0.0,0.00000,0.000000e+00,0.00000
2020-04-30,482.0,5778.0,67002.0,81297.0,1.056920e+06,66003.0,1.037903e+06,31.40952,495.46769,1.73039,...,9.898430e+05,4.640230e+02,0.000000e+00,0.00000,0.000000e+00,0.00000,0.0,0.00000,0.000000e+00,0.00000
2020-05-31,633.0,23385.0,511377.0,428975.0,8.005279e+06,508063.0,7.929984e+06,239.72561,3752.75061,2.03386,...,1.698881e+07,7.964091e+03,0.000000e+00,0.00000,0.000000e+00,0.00000,0.0,0.00000,0.000000e+00,0.00000
2020-06-30,744.0,30416.0,1358162.0,896462.0,2.833123e+07,1353418.0,2.816292e+07,636.68528,13281.24019,1.47260,...,6.395057e+07,2.997903e+04,0.000000e+00,0.00000,0.000000e+00,0.00000,0.0,0.00000,0.000000e+00,0.00000
2020-07-31,903.0,32924.0,2375783.0,1257680.0,6.285490e+07,2370476.0,6.264186e+07,1113.73025,29465.40394,1.18192,...,1.570695e+08,7.363177e+04,0.000000e+00,0.00000,0.000000e+00,0.00000,0.0,0.00000,0.000000e+00,0.00000
2020-08-31,1041.0,28853.0,3349745.0,1243672.0,1.031060e+08,3345605.0,1.029039e+08,1570.30848,48334.47268,1.01053,...,3.088827e+08,1.447994e+05,0.000000e+00,0.00000,0.000000e+00,0.00000,0.0,0.00000,0.000000e+00,0.00000
2020-09-30,1138.0,22469.0,4008406.0,902811.0,1.322408e+08,4005246.0,1.320948e+08,1879.07856,61992.45296,0.90986,...,4.143522e+08,1.942419e+05,0.000000e+00,0.00000,0.000000e+00,0.00000,0.0,0.00000,0.000000e+00,0.00000
2020-10-31,1310.0,15890.0,4736028.0,720306.0,1.611664e+08,4733727.0,1.610360e+08,2220.17644,75552.28568,0.91127,...,5.232688e+08,2.453003e+05,0.000000e+00,0.00000,0.000000e+00,0.00000,0.0,0.00000,0.000000e+00,0.00000


In [ ]:
round(months.describe(),2)

,epi_week,newDeaths,deaths,newCases,totalCases,deathsMS,totalCasesMS,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,...,tests,tests_per_100k_inhabitants,vaccinated,vaccinated_per_100_inhabitants,vaccinated_second,vaccinated_second_per_100_inhabitants,vaccinated_single,vaccinated_single_per_100_inhabitants,vaccinated_third,vaccinated_third_per_100_inhabitants
count,34.00,34.00,34.00,34.00,3.400000e+01,34.00,3.400000e+01,34.00,34.00,34.00,...,3.400000e+01,34.00,3.400000e+01,34.00,3.400000e+01,34.00,3.400000e+01,34.00,3.400000e+01,34.00
mean,2332.56,20254.59,12006225.29,1026711.71,5.002373e+08,12003608.06,5.001031e+08,5628.33,234503.49,0.82,...,1.287302e+09,603467.29,2.519963e+09,1181.32,1.993976e+09,934.74,7.352526e+07,34.47,8.798319e+08,412.45
std,1275.29,20397.84,7856142.06,835901.42,3.629144e+08,7856950.36,3.629408e+08,3682.84,170128.63,0.37,...,8.140693e+08,381623.05,2.423381e+09,1136.04,2.150183e+09,1007.97,7.655157e+07,35.89,1.313638e+09,615.81
min,45.00,0.00,0.00,2.00,7.000000e+00,0.00,5.000000e+00,0.00,0.00,0.00,...,0.000000e+00,0.00,0.000000e+00,0.00,0.000000e+00,0.00,0.000000e+00,0.00,0.000000e+00,0.00
25%,1332.50,4917.75,4799252.75,393872.75,1.651269e+08,4797046.25,1.649930e+08,2249.82,77408.92,0.65,...,5.590938e+08,262094.48,0.000000e+00,0.00,0.000000e+00,0.00,0.000000e+00,0.00,0.000000e+00,0.00
50%,2279.00,14592.50,14142449.00,879654.50,5.083881e+08,14135000.50,5.082470e+08,6629.76,238324.44,0.82,...,1.513731e+09,709613.57,2.183504e+09,1023.59,8.384701e+08,393.06,3.886985e+07,18.22,1.000960e+05,0.05
75%,3300.50,29384.00,19243520.00,1351001.50,7.594539e+08,19242226.25,7.590293e+08,9021.06,356020.21,0.87,...,2.047232e+09,959710.77,5.040548e+09,2362.93,4.340861e+09,2034.93,1.537969e+08,72.10,1.646595e+09,771.90
max,4544.00,82364.00,21304378.00,3322951.00,1.077397e+09,21304378.00,1.077397e+09,9987.16,505067.07,2.03,...,2.198620e+09,1030678.79,5.640051e+09,2643.97,5.173237e+09,2425.13,1.684772e+08,78.98,3.682813e+09,1726.45


In [11]:
fig = px.line(months, x=months.index, y="newCases", title='Número de novos casos divulgados',
             labels = {
                "newCases": "Quantidade",
                "date": "Data"
                })
fig.show()

NameError: name 'months' is not defined

In [46]:
fig = px.line(months, x=months.index, y="newDeaths", title='Número de Novos Óbitos Divulgados',
             labels = {
                "deathsMS": "Quantidade",
                "date": "Data"
                })
fig.show()

In [47]:
months["deaths_var"] = months["newDeaths"].pct_change()

fig = px.line(months, x=months.index, y=months["deaths_var"], title='Variação no Número de Óbitos por Covid',
             labels = {
                "deaths_var": "Variação",
                "date": "Data"
                })
fig.show()

Por apresentar valores iniciais muito altos - até mesmo valores infinitos - a análise da variação no número de óbitos é iniciada a partir do terceiro mês de pandemia (abril), conforme apresentado no gráfico.

## Obtendo o IPCA do período

Base de dados disponível em:

<https://dadosabertos.bcb.gov.br/dataset/4449-indice-nacional-de-precos-ao-consumidor-amplo-ipca---precos-monitorados---total>

In [48]:
ipca = requests.get("https://api.bcb.gov.br/dados/serie/bcdata.sgs.4449/dados?formato=csv")
ipca

<Response [200]>

In [49]:
ipca = ipca.content.decode("utf-8")
ipca = [x.split(';') for x in ipca.split('\r\n')]
ipca

[['"data"', '"valor"'],
 ['"01/01/1992"', '"28,50"'],
 ['"01/02/1992"', '"29,94"'],
 ['"01/03/1992"', '"22,48"'],
 ['"01/04/1992"', '"19,35"'],
 ['"01/05/1992"', '"19,88"'],
 ['"01/06/1992"', '"19,46"'],
 ['"01/07/1992"', '"24,11"'],
 ['"01/08/1992"', '"26,60"'],
 ['"01/09/1992"', '"21,26"'],
 ['"01/10/1992"', '"21,46"'],
 ['"01/11/1992"', '"17,31"'],
 ['"01/12/1992"', '"24,51"'],
 ['"01/01/1993"', '"32,93"'],
 ['"01/02/1993"', '"29,24"'],
 ['"01/03/1993"', '"24,51"'],
 ['"01/04/1993"', '"29,15"'],
 ['"01/05/1993"', '"29,48"'],
 ['"01/06/1993"', '"30,16"'],
 ['"01/07/1993"', '"35,26"'],
 ['"01/08/1993"', '"35,30"'],
 ['"01/09/1993"', '"36,14"'],
 ['"01/10/1993"', '"34,63"'],
 ['"01/11/1993"', '"35,47"'],
 ['"01/12/1993"', '"36,16"'],
 ['"01/01/1994"', '"37,74"'],
 ['"01/02/1994"', '"42,49"'],
 ['"01/03/1994"', '"41,27"'],
 ['"01/04/1994"', '"42,14"'],
 ['"01/05/1994"', '"45,22"'],
 ['"01/06/1994"', '"42,41"'],
 ['"01/07/1994"', '"4,64"'],
 ['"01/08/1994"', '"-0,12"'],
 ['"01/09/1994"',

In [50]:
# Removendo as aspas ("") prensentes em cada item do dataframe:
for row in range(0, len(ipca)):
    for item in range(0, len(ipca[row])):
        ipca[row][item] = ipca[row][item].replace('\"', "")

In [51]:
ipca_df = pd.DataFrame(ipca[1:], columns= ipca[0])
ipca_df

,data,valor
0,01/01/1992,"28,50"
1,01/02/1992,"29,94"
2,01/03/1992,"22,48"
3,01/04/1992,"19,35"
4,01/05/1992,"19,88"
...,...,...
366,01/07/2022,"-4,35"
367,01/08/2022,"-2,55"
368,01/09/2022,"-1,21"
369,01/10/2022,"0,18"


In [52]:
# Removendo a última linha, que possui valores nulos:
ipca_df = ipca_df[:-1]

In [53]:
# Colocando a data no formato datetime:
ipca_df["data"] = pd.to_datetime(ipca_df["data"], format="%d/%m/%Y")

C:\Users\Giga\AppData\Local\Temp\ipykernel_36372\4253406373.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [54]:
# Trocando os pontos por vírgulas:
ipca_df["valor"] = [row.replace(",", ".") for row in ipca_df["valor"]]

C:\Users\Giga\AppData\Local\Temp\ipykernel_36372\2544237049.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [55]:
# Colocando o valor no formato float:
ipca_df["valor"] = ipca_df["valor"].astype(float)

C:\Users\Giga\AppData\Local\Temp\ipykernel_36372\2883942928.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [56]:
# Separando um dataframe para analizar o IPCA entre 2020 e os dias atuais:
ipca = ipca_df.loc[ipca_df["data"]>"2020-04-01"]

In [57]:
ipca

,data,valor
340,2020-05-01,-1.02
341,2020-06-01,0.89
342,2020-07-01,1.23
343,2020-08-01,0.78
344,2020-09-01,0.13
345,2020-10-01,0.23
346,2020-11-01,0.41
347,2020-12-01,2.04
348,2021-01-01,-0.29
349,2021-02-01,1.69


In [58]:
round(ipca.describe(),2)

,valor
count,30.00
mean,0.52
std,1.51
min,-4.35
25%,0.07
50%,0.52
75%,1.60
max,2.81


In [59]:
fig = px.line(ipca, x="data", y="valor", title='Variação do IPCA no Brasil',\
             labels = {
                "valor": "Variação %",
                "data": "Data"
                })
fig.show()

## Obtendo a Cotação da Bolsa de Valores e do Dolar para o período

Através da biblioteca yfinance

In [60]:
ibov = yf.download('^bvsp', start='2020-03-01', end="2022-11-01", interval='1mo')
usd = yf.download('USDBRL=X', start='2020-03-01', end="2022-11-01", interval='1mo')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [61]:
# Obtendo as variações mensais da bolsa de valores:
ibov = ibov.pct_change()
usd = usd.pct_change()

display(ibov.head(), usd.head())

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-03-01,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-01,-0.299722,-0.231664,0.099058,0.102520,0.102520,-0.225514
2020-05-01,0.102587,0.053745,0.116442,0.085671,0.085671,0.051728
2020-06-01,0.085639,0.109001,0.147166,0.087560,0.087560,0.054299
2020-07-01,0.087728,0.082002,0.094718,0.082646,0.082646,-0.087237


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-03-01,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-01,0.164051,0.096536,0.137055,0.027365,0.027365,NaN
2020-05-01,0.054209,0.044278,0.043352,-0.000469,-0.000469,NaN
2020-06-01,-0.027543,-0.077380,-0.085484,0.013090,0.013090,NaN
2020-07-01,0.023712,-0.007127,0.056944,-0.045945,-0.045945,NaN


In [62]:
# dict for the dataframes and their names
dfs = {"ipca" : ipca, "covid": months, "ibov":ibov, "usd":usd}

# plot the data
fig = go.Figure()
fig = fig.add_trace(go.Scatter(x = dfs["ipca"]["data"],
                               y = dfs["ipca"]["valor"], 
                               name = "Variação do IPCA no Brasil"))

fig = fig.add_trace(go.Scatter(x = dfs["covid"].index[:-2],
                               y = dfs["covid"]["deaths_var"], 
                               name = "Variação no Número de Óbitos por Covid"))

fig = fig.add_trace(go.Scatter(x = dfs["ibov"].index[1:-1],
                               y = dfs["ibov"]["Close"], 
                               name = "Variação na Bolsa de Valores"))

fig = fig.add_trace(go.Scatter(x = dfs["usd"].index[1:],
                               y = dfs["usd"]["Close"], 
                               name = "Variação da Cotação do Dolar"))

fig.show()

In [63]:
len(months["deaths_var"])

34

In [64]:
len(ipca["valor"])

30

In [65]:
len(ibov["Close"])

33

In [66]:
pd.DataFrame({
    "ipca": [row for row in ipca["valor"]],
    "covid": [row for row in months["deaths_var"][2:-2]],
    "ibov": [row for row in ibov["Close"][1:-2]],
    "usd": [row for row in usd["Close"][1:-1]]
    
})

,ipca,covid,ibov,usd
0,-1.02,27.603960,0.102520,0.027365
1,0.89,3.047248,0.085671,-0.000469
2,1.23,0.300663,0.087560,0.013090
3,0.78,0.082457,0.082646,-0.045945
4,0.13,-0.123648,-0.034427,0.045810
5,0.23,-0.221259,-0.047963,0.044620
6,0.41,-0.292803,-0.006881,0.026161
7,2.04,-0.163310,0.158975,-0.075340
8,-0.29,0.639413,0.095676,-0.028388
9,1.69,0.356258,-0.027652,0.052007


In [67]:
corr_df = pd.DataFrame({
    "ipca": [row for row in ipca["valor"]],
    "covid": [row for row in months["deaths_var"][2:-2]],
    "ibov": [row for row in ibov["Close"][1:-2]],
    "usd": [row for row in usd["Close"][1:-1]]
    
})

In [68]:
corr_df.corr()

,ipca,covid,ibov,usd
ipca,1.000000,-0.187565,0.069953,-0.338606
covid,-0.187565,1.000000,0.292992,0.092094
ibov,0.069953,0.292992,1.000000,-0.648904
usd,-0.338606,0.092094,-0.648904,1.000000
